In [42]:
import pandas as pd
import numpy as np

In [ ]:
df_og = pd.read_csv('/content/drive/MyDrive/elections-2025/results_pct_20251104.txt', sep='\t')

### MAKE SURE YOU REMOVE THIS ON ELECTION NIGHT ####
df_og['Total Votes'] = np.random.randint(0, 200, size=len(df_og))
### MAKE SURE YOU REMOVE THIS ON ELECTION NIGHT ####

df = df_og 

#Calculate total votes per contest
df['contest_total'] = df.groupby('Contest Name')['Total Votes'].transform('sum')
#Calculate candidate total within each contest
#This groups by Contest Name AND Choice to ensure that any 'choice' that has the same name in two different contests don't get counted togeter
#Such as "for" or "against" or Write in
df['candidate_total'] = df.groupby(['Contest Name','Choice'])['Total Votes'].transform('sum')
#Calculate the percent that each candidate got for their race
df['Percent of the Vote'] = ((df['candidate_total']/df['contest_total'])*100).round(2)
#Clean the Choice Party column, strip spaces
df['Choice Party'] = df['Choice Party'].str.strip()
#Select only for the counties that we are care about in our coverage area
counties = ['MECKLENBURG', 'GASTON', 'IREDELL', 'CABARRUS', 'UNION']
df = df[df['County'].isin(counties)].copy()
#add party to the choice name if they have a party, if they don't have a party, skip it
df['choice'] = np.where(
    df['Choice Party'].fillna('').str.strip() == '',
    df['Choice'],
    df['Choice'] + ' (' + df['Choice Party'] + ')'
)

#convert the column indicating how many choices you can pick to a string so I can attach that to the contest name for the viz
df['Vote For'] = df['Vote For'].astype(str)

#attach vote for to the contest name 
df['Contest Name'] = df['Contest Name']+ ' (Vote for ' +df['Vote For'] + ')'
#boil down data frame to columns that are unique to each contest
df = df[['Contest Name', 'County', 'Vote For', 'choice', 'Choice Party', 'candidate_total', 'contest_total', 'Percent of the Vote']].copy()
#format the Contest Name so it is not shouting at people who look at my data viz
df['Contest Name'] = df['Contest Name'].str.title()

#drop the extra columns that are associated with precinct-level results
df = df.drop_duplicates()
#kill out Miscellaneous for Party because it offers nothing in the viz
df['choice'] = df['choice'].str.replace(r'\(Miscellaneous\)', '', regex=True).str.strip()


#create viz for CLT council and mayor
cltcouncil = df[
    (df['Contest Name'].str.contains('Charlotte City Council')) |
    (df['Contest Name'].str.contains('City Of Charlotte Mayor'))
].copy()

cltcouncil['order'] = np.where(cltcouncil['Contest Name']=='City Of Charlotte Mayor (Vote For 1)', "A", "B")
cltcouncil = cltcouncil.sort_values('order')
cltcouncil.to_excel('/content/drive/MyDrive/elections-2025/clt-city-council.xlsx', index=False)


#MECKLENBURG
meck = df[df['County'].str.contains('MECKLENBURG')].copy()
meck['order'] = np.where(meck['Contest Name']=='City Of Charlotte Mayor (Vote For 1)', "A", "B")
meck = meck.sort_values('order')
meck.to_excel('/content/drive/MyDrive/elections-2025/mecklenburg.xlsx', index=False)

#transit tax

transit = df[df['Contest Name']=='Mecklenburg County Public Transportation Sales And Use Tax (Vote For 1)']
transit.to_excel('/content/drive/MyDrive/elections-2025/transit.xlsx', index=False)

#GASTON
gaston = df[df['County'].str.contains('GASTON')]
gaston.to_excel('/content/drive/MyDrive/elections-2025/gaston.xlsx', index=False)

#IREDELL
IREDELL = df[df['County'].str.contains('IREDELL')]
IREDELL.to_excel('/content/drive/MyDrive/elections-2025/IREDELL.xlsx', index=False)

#CABARRUS
CABARRUS = df[df['County'].str.contains('CABARRUS')]
CABARRUS.to_excel('/content/drive/MyDrive/elections-2025/CABARRUS.xlsx', index=False)

#UNION
UNION = df[df['County'].str.contains('UNION')]
UNION.to_excel('/content/drive/MyDrive/elections-2025/UNION.xlsx', index=False)